#미국 경제 뉴스 크롤링
https://www.investing.com/news/economy


1. 해당 기사들 페이지 링크는

 첫번째 페이지 : https://www.investing.com/news/economy

 두번째 페이지 : https://www.investing.com/news/economy/2

 세번째 페이지 : https://www.investing.com/news/economy/3

 이런 식으로 2번째 페이지부터 숫자가 붙음.

2. 기사 링크를 포함하는 a 태그의 클래스가 text-inv-blue-500 임

3. 기사 본문 포함하는 'class': 'article_articlePage__UMz3q'

4. robots.txt를 통해 크롤링 가능한지 확인함
 이외의 경로는 명시적으로 허용되어 있거나 금지되지 않았으므로, 경제 뉴스 기사 경로(/news/economy)는 크롤링할 수 있음.


In [2]:
!pip install requests beautifulsoup4

In [27]:
# 라이브러리 임포트
import requests
from bs4 import BeautifulSoup
import csv

# 기본 URL 정의
BASE_URL = "https://www.investing.com"
ECONOMY_NEWS_URL = f"{BASE_URL}/news/economy"

# 기사 링크를 가져오는 함수
def get_article_links(page_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(page_url, headers=headers)
    if response.status_code != 200:
        print(f"페이지를 가져오지 못했습니다: {page_url}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('a', {'data-test': 'article-title-link'})
    links = [BASE_URL + article['href'] for article in articles if article['href']]

    if not links:
        print(f"이 페이지에서 기사 링크를 찾을 수 없습니다: {page_url}")

    return links

# 기사를 스크래핑하는 함수 (링크만 수집)
def scrape_investing_news(start_page, end_page):
    all_links = []
    for page in range(start_page, end_page + 1):
        if page == 1:
            page_url = ECONOMY_NEWS_URL
        else:
            page_url = f"{ECONOMY_NEWS_URL}/{page}"

        article_links = get_article_links(page_url)
        all_links.extend(article_links)
        print(f"{end_page} 페이지 중 {page} 페이지 스크래핑 완료, {len(article_links)} 개 링크 발견")
    return all_links

# 링크를 CSV 파일로 저장하는 함수
def save_links_to_csv(links, filename):
    if not links:
        print("저장할 링크가 없습니다.")
        return

    with open(filename, 'w', newline='', encoding='utf-8') as output_file:
        writer = csv.writer(output_file)
        writer.writerow(["link"])
        for link in links:
            writer.writerow([link])
    print(f"{len(links)} 개 링크를 {filename} 파일로 저장했습니다.")

# 실행
start_page = int(input("시작 페이지 입력: "))
end_page = int(input("끝 페이지 입력: "))

links = scrape_investing_news(start_page, end_page)
output_csv = 'investing_links.csv'
save_links_to_csv(links, output_csv)

# CSV 파일 다운로드
from google.colab import files
files.download(output_csv)


시작 페이지 입력: 6
끝 페이지 입력: 10
10 페이지 중 6 페이지 스크래핑 완료, 35 개 링크 발견
10 페이지 중 7 페이지 스크래핑 완료, 35 개 링크 발견
10 페이지 중 8 페이지 스크래핑 완료, 35 개 링크 발견
10 페이지 중 9 페이지 스크래핑 완료, 35 개 링크 발견
10 페이지 중 10 페이지 스크래핑 완료, 35 개 링크 발견
175 개 링크를 investing_links.csv 파일로 저장했습니다.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**링크 csv 파일 통해서 본문 가져오기**

In [28]:
import requests
from bs4 import BeautifulSoup
import csv

# 기사 본문을 가져오는 함수
def get_investing_news_content(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        content = soup.find('div', {'class': 'article_articlePage__UMz3q'})
        if content:
            # 본문에서 스크립트, 스타일, 불필요한 태그 제거
            for unwanted in content(['script', 'style', 'div', 'span', 'figure', 'br']):
                unwanted.decompose()

            text = content.get_text(separator=' ', strip=True)

            return text.strip()
        else:
            return "본문을 찾을 수 없습니다."
    else:
        return "페이지를 불러올 수 없습니다."

# 링크가 포함된 CSV 파일에서 링크를 읽고, 본문을 추출하여 새로운 CSV 파일에 저장하는 함수
def extract_content_from_links(input_csv, output_csv):
    with open(input_csv, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        articles = []
        for row in reader:
            url = row['link']
            content = get_investing_news_content(url)
            articles.append({'link': url, 'content': content})
            print(f"Extracted content from {url}")

    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['link', 'content']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(articles)
    print(f"Extracted contents saved to {output_csv}")

# 실행
input_csv = 'investing_links.csv'
output_csv = 'investing_articles.csv'
extract_content_from_links(input_csv, output_csv)

# CSV 파일 다운로드
from google.colab import files
files.download(output_csv)


Extracted content from https://www.investing.com/news/economy-news/japans-top-council-urges-govt-boj-to-guide-policy-with-eye-on-weak-yen-3540512
Extracted content from https://www.investing.com/news/commodities-news/fires-in-canadas-jasper-town-coming-under-control-say-authorities-3540331
Extracted content from https://www.investing.com/news/economy-news/australias-banking-regulator-says-it-will-not-ease-home-lending-rules-3540441
Extracted content from https://www.investing.com/news/economic-indicators/peru-2024-gdp-growth-should-surpass-31-president-says-3540416
Extracted content from https://www.investing.com/news/economy-news/analysisbojs-victory-lap-on-deflation-paves-way-for-ratehike-cycle-3540430
Extracted content from https://www.investing.com/news/economy/top-5-things-to-watch-in-markets-in-the-week-ahead-3540350
Extracted content from https://www.investing.com/news/commodities-news/fire-smoke-upend-western-canadas-summer-tourism-season-3540312
Extracted content from https://

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>